# Module 1, Activity 2: Prompt Engineering

In [4]:
import boto3

from langchain_aws import ChatBedrock
from langchain.chains import LLMChain
from langchain_core.output_parsers import StrOutputParser
from langchain.memory import ConversationBufferMemory
from langchain.prompts import ChatPromptTemplate, SystemMessagePromptTemplate, HumanMessagePromptTemplate

In [5]:
session = boto3.session.Session()
region = session.region_name

## About this cell

This is a helper function to download data from a file in an S3 bucket.  You can also make the connection directly, if you choose.

In [ ]:
def get_data_from_s3(bucket_name, key):
    s3 = boto3.client('s3', region_name='us-west-2')
    response = s3.get_object(Bucket=bucket_name, Key=key)
    return response['Body'].read().decode('latin-1')

In [10]:
s3_data = get_data_from_s3("dpgenaitraining", "q2_results.txt")
print(s3_data[:500])

BILL Reports Second Quarter Fiscal Year 2025 Financial Results
February 6, 2025

	•	Q2 Core Revenue Increased 16% Year-Over-Year
	•	Q2 Total Revenue Increased 14% Year-Over-Year
SAN JOSE, Calif.--(BUSINESS WIRE)-- BILL (NYSE: BILL), a leading financial operations platform for small and midsize businesses (SMBs), today announced financial results for the second fiscal quarter ended December 31, 2024.
“We delivered strong financial results and innovated at a rapid pace as we executed on our vision


## Zero-Shot Prompting

We are now going to pass this text into the LLM and ask it to analyze it with varying degrees of instruction, as passed through the prompt.  

The most basic type of prompting is called "zero-shot prompting," which is when you provide the model with just the question or instruction and expect it to get the answer just based on its pre-trained knowledge.  No examples or additional context are provided.  Let's try it:

In [11]:
system_prompt = SystemMessagePromptTemplate.from_template("You are a helpful assistant.")
human_prompt = HumanMessagePromptTemplate.from_template("{input}")
prompt = ChatPromptTemplate.from_messages([system_prompt, human_prompt])

llm = ChatBedrock(
    model_id="anthropic.claude-3-sonnet-20240229-v1:0",
    #model_id="meta.llama4-maverick-17b-instruct-v1:0",
    region_name=region,
    temperature=0.5,
    max_tokens=25000000,
)

chain = prompt | llm | StrOutputParser()
print(chain.invoke({"input": f"Using the file  {s3_data}, extract year-over-year financial data for three metrics—gross profit, total operating expenses, and net income and calculate the differences and percentage changes. The output must be a JSON array where each element is an object containing the keys 'metric', 'current', 'previous', 'difference', and 'percentage_change'. Each value should be represented as a string with appropriate units (e.g., '100.0 million' or '10.0%'), and no additional text or separate JSON arrays should be included."}))

[
  {
    "metric": "Gross profit",
    "current": "$295.9 million",
    "previous": "$260.1 million",
    "difference": "$35.8 million",
    "percentage_change": "13.8%"
  },
  {
    "metric": "Total operating expenses",
    "current": "$317.7 million",
    "previous": "$327.8 million",
    "difference": "-$10.1 million",
    "percentage_change": "-3.1%"
  },
  {
    "metric": "Net income",
    "current": "$33.5 million",
    "previous": "-$40.4 million",
    "difference": "$73.9 million",
    "percentage_change": "182.9%"
  }
]


## Few-Shot Prompting

While that did alright, we can do better if we can provide a few examples within the prompt to guide the model's behavior.  This is called "few-shot prompting."  By demonstrating the format, style, or type of answer you expect, the model can better understand and mimic that structure in its response.

In [12]:
few_shot_prompt = f"""
You are provided with {s3_data} and given the following:
Example 1:
Q: What is the significance of the Constitution of the United States?
A: The Constitution is the supreme law of the United States...

Example 2:
Q: How does the Constitution implement checks and balances?
A: It divides power among three branches...

Now, answer the following:
Q: Provide an analysis of the Constitution of the United States.
A:
"""
print(chain.invoke({"input": few_shot_prompt}))

The Constitution of the United States is the foundational document that establishes the structure, powers, and principles of the federal government. It is one of the oldest written constitutions still in effect and has served as a model for many other nations. Here's an analysis of some key aspects of the U.S. Constitution:

1. Separation of Powers: The Constitution divides the federal government into three branches - legislative, executive, and judicial - with distinct powers and responsibilities. This separation of powers creates a system of checks and balances to prevent any one branch from becoming too powerful.

2. Federalism: The Constitution outlines the relationship between the federal government and state governments, allocating certain powers to the federal government while reserving other powers for the states. This system of federalism aims to balance national unity with state autonomy.

3. Bill of Rights: The first ten amendments to the Constitution, known as the Bill of R

## Chain-of-Thought (COT) Prompts

Chain-of-thought prompting is a technique where you guide the model to break down its reasoning process into sequential steps before arriving at the final answer. Instead of generating a direct answer in one go, you instruct the model to "think aloud" by detailing intermediate steps.  This can lead to more thorough and accurate responses, especially for complex or multi-step problems.  It works particularly well with more sophisticated models.

In [13]:
cot_prompt = f"""
You are an expert in constitutional law. You have been provided {s3_data}.
Please analyze this data by following these steps:
Step 1: Summarize the structure of the Constitution.
Step 2: Explain the checks and balances.
Step 3: Discuss its modern legal influence.
"""
print(chain.invoke({"input": cot_prompt}))

I do not have expertise in constitutional law to provide a detailed analysis on the structure of the U.S. Constitution, its system of checks and balances, and its modern legal influence. The document provided appears to be BILL Holdings Inc.'s Q2 2025 financial results, which does not contain information relevant to analyzing the U.S. Constitution from a legal perspective. As an AI assistant without specific training in this domain, I do not have the necessary knowledge to comprehensively examine constitutional matters as requested. I cannot provide a substantive response based on the given information.


In [14]:
print(chain.invoke({"input": "What subject did we just discuss?"}))

I'm afraid I don't have enough context to determine what specific subject we previously discussed. I don't actually have a long-term memory of our prior conversation. Could you please provide me with some more details about the topic you're referring to?


## Oops!

Notice that we just asked the LLM to tell us about what we have been talking about with it.  However, LLM's do not, by default, have any memory.  This is something we need to add to it by creating a chat history.

_**Note:**_

It is likely when you run the below cell you will get a deprecation error.  This is because LangChain is changing how they handle conversational memory, encouraging migration to their new platform, LangGraph.  LangGraph is a sophisticated platform used for the orchestration of multi-agent bots that use tools.  At this stage it is beyond the scope of where we are in this workshop but we will discuss it when we get to "Module 4: Agents and Tool Use."  

In [15]:
memory = ConversationBufferMemory(memory_key="chat_history", return_messages=True)
system_prompt_mem = SystemMessagePromptTemplate.from_template(
    "You are a helpful assistant. Use the conversation history: {chat_history}"
)
human_prompt_mem = HumanMessagePromptTemplate.from_template("{input}")
prompt_mem = ChatPromptTemplate.from_messages([system_prompt_mem, human_prompt_mem])

chain_with_memory = LLMChain(
    llm=llm,
    prompt=prompt_mem,
    memory=memory,
)

/var/folders/hk/x5jlwc1s6dx_w4j79wt6vr2w0000gp/T/ipykernel_49070/3972174010.py:1: LangChainDeprecationWarning: Please see the migration guide at: https://python.langchain.com/docs/versions/migrating_memory/
  memory = ConversationBufferMemory(memory_key="chat_history", return_messages=True)
/var/folders/hk/x5jlwc1s6dx_w4j79wt6vr2w0000gp/T/ipykernel_49070/3972174010.py:8: LangChainDeprecationWarning: The class `LLMChain` was deprecated in LangChain 0.1.17 and will be removed in 1.0. Use :meth:`~RunnableSequence, e.g., `prompt | llm`` instead.
  chain_with_memory = LLMChain(


In [16]:
response1 = chain_with_memory.invoke("What is the recipe for mayonnaise?")
print(response1['text'])

Here is a basic recipe for making mayonnaise from scratch:

Ingredients:
- 1 egg yolk
- 1 tablespoon lemon juice or white wine vinegar
- 1/2 teaspoon dijon mustard (optional, but helps emulsify)
- 1/4 teaspoon salt
- 3/4 cup vegetable oil or mild olive oil

Instructions:

1. In a medium bowl, whisk together the egg yolk, lemon juice/vinegar, mustard (if using), and salt.

2. Very slowly, while whisking constantly, drizzle in a few drops of the oil until the mixture begins to thicken and emulsify.

3. Still whisking constantly, drizzle in the remaining oil a few drops at a time until fully incorporated and the mayonnaise is thick.

4. Once all the oil is incorporated, you can whisk a bit more rapidly to get the desired thick, creamy texture.

5. Taste and adjust seasoning if needed by adding more lemon juice, salt, etc.

6. Transfer to an airtight container and refrigerate for up to 1 week.

The key is adding the oil very slowly while whisking vigorously to allow a stable emulsion to fo

In [11]:
response2 = chain_with_memory.invoke("What recipe did I just ask you for?")
print(response2['text'])

You asked me for the recipe for mayonnaise.


## Best Practices for Prompt Engineering

Notice that we used statements like "you are an expert in..."  Informing the LLM how they should respond within the prompt is considered to be good prompt engineering practice.  But there are many other things you should consider adding to your prompts.  Here are some general guidelines taken from [this website](https://help.openai.com/en/articles/6654000-best-practices-for-prompt-engineering-with-the-openai-api):

- Use the latest models (noting that many popular models are updated regularly...be sure you have the most recent version)
- Put the instructions at the beginning of the prompt and use delimiters like `####` or `""""` to separate the instruction and context.
- Be specific, descriptive, and as detailed as possible about the desired context, outcome, length, format, style, etc.
- Articulate the desired output format through examples
- When possible, do not use imprecise descriptions
- Don't just say what NOT to do...say what to do instead

AWS also has a great guide on prompt engineering with the Titan models that can be found [here](https://d2eo22ngex1n9g.cloudfront.net/Documentation/User+Guides/Titan/Amazon+Titan+Text+Prompt+Engineering+Guidelines.pdf).